<a href="https://colab.research.google.com/github/sollall/myidea/blob/master/%E5%88%86%E5%AD%90%E9%87%8F%E5%88%86%E5%B8%83_FFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [3]:
import cmath
 
class FFT:
    def __init__(self):
        self.pi = cmath.pi
    
    def convolution(self, g, h):
        l = len(g) + len(h)
        n = 1 << l.bit_length()
        
        g = g + [0] * (n - len(g))
        h = h + [0] * (n - len(h))
       
        gg = self.fft(g, n)
        hh = self.fft(h, n)
        ff = [gg[i] * hh[i] for i in range(n)]
 
        return self.inv_fft(ff, n)
 
    def fft(self, f, n, sgn = 1):
        if n == 1: return f
 
        f0 = [f[2 * i] for i in range(n // 2)]
        f1 = [f[2 * i + 1] for i in range(n // 2)]
        f0 = self.fft(f0, n // 2, sgn)
        f1 = self.fft(f1, n // 2, sgn)
 
        zeta = cmath.rect(1, sgn * self.pi * 2 / n)
        pow_zeta = 1
 
        for i in range(n):
            f[i] = f0[i % (n // 2)] + pow_zeta * f1[i % (n // 2)]
            pow_zeta *= zeta
 
        return f
 
    def inv_fft(self, f, n):
        ret = self.fft(f, n, -1)
        for i in range(n):
            ret[i] = (ret[i].real / n)
 
        return ret

In [ ]:
Time=15000
N=3*10**4
mol_init=5000
r_init=1#1stepでの反応分子数　途中で小さくなる

mols=[0 for i in range(N)]
mols[1]=mol_init

A=mols[:]
M_init=sum(mols)
SUM=sum(A)

fft=FFT()

k=0.005
r=0
threshold=r_init*k

for step in range(Time):
    r_init,r=(1-k)*r_init,r+k*r_init
    print(step)

    SUM_A=sum(A)
    new_A=[A[i]-A[i]/SUM_A*r for i in range(len(A))]
    
    C=(fft.convolution(A,A))
    SUM_C=sum(C)

    C=[C[i]*r/2/SUM_C for i in range(len(C))]
    temp=[new_A[i]+C[i] for i in range(len(A))]

    new_A=temp[:]
    r*=(1-r/SUM_A)
    print(new_A[:30])
    print(C[:30])
    print(sum(new_A),sum(A))
    print("r:"+str(r))
    
    if mol_init-sum([new_A[i]*i for i in range(len(new_A))])>1:
        break

    A=new_A[:]

    
    plt.plot(A)
    plt.show()
    plt.plot([A[i]*i for i in range(len(A))])
    plt.show()

    
    
